1. Tuner with a simple network with few layers

In [1]:
# Import thư viên
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, BatchNormalization
from tensorflow.keras.layers import ReLU, MaxPool2D, AvgPool2D, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [2]:

# Load data
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize
img_train = img_train.astype("float32")/255
img_test = img_test.astype("float32")/255

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 3us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
# Create function build Hyper Model
def model_builder(hp):
  model = keras.Sequential()
  model.add(Flatten(input_shape=(28,28))) # layer 1
  X = hp.Int("units", min_value=32, max_value=512, step=32) # parameter tuning 
  model.add(Dense(units=X, activation="relu"))
  model.add(Dense(10))

  lr = hp.Choice("lr",values=[1e-2,1e-3,1e-4]) # lr parameter

  model.compile(optimizer=Adam(learning_rate=lr), loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
  return model

In [4]:
# Init Tuner
tuner = kt.Hyperband(model_builder, objective="val_accuracy", directory="tuner_dir_01", project_name="Demo_Tuner")

# Keras Callbacks - Early Stop
stop_early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

f:\test\chatbot\env\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
# Bắt đầu search xem bộ giá trị HP nào tốt nhất
tuner.search( img_train, label_train, validation_split=0.2, callbacks=[stop_early])

Trial 46 Complete [00h 00m 05s]
val_accuracy: 0.8511666655540466

Best val_accuracy So Far: 0.8737499713897705
Total elapsed time: 00h 03m 25s


In [28]:

# Print hyper parameters
best_hps = tuner.get_best_hyperparameters()[0]

print("Best units = ", best_hps.get('units'), "Optimal learning rate = ", best_hps.get('lr'))

Best units =  480 Optimal learning rate =  0.001


In [8]:
# Build model with best hyper parameters
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(img_train, label_train, epochs=20, validation_split=0.2) # Hàm train model thông thường

best_model.save("model.h5")

Epoch 1/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7844 - loss: 0.6167 - val_accuracy: 0.8572 - val_loss: 0.4074
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8626 - loss: 0.3792 - val_accuracy: 0.8727 - val_loss: 0.3560
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8786 - loss: 0.3286 - val_accuracy: 0.8775 - val_loss: 0.3407
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8871 - loss: 0.3046 - val_accuracy: 0.8758 - val_loss: 0.3460
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8944 - loss: 0.2850 - val_accuracy: 0.8842 - val_loss: 0.3224
Epoch 6/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8996 - loss: 0.2700 - val_accuracy: 0.8757 - val_loss: 0.3477
Epoch 7/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9038 - loss: 0.2587 - val_accuracy: 0.8882 - val_loss: 0.3187
Epoch 8/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9078 - loss: 0.2496 - 

In [13]:
# Get best model
best_model = tuner.get_best_models()[0]
# best_model.build(img_train.shape)
best_model.summary()

best_model.fit(img_train, label_train, epochs=20, validation_split=0.2) # Hàm train thông thường
best_model.save("model.h5")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 480)            │       376,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         4,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 381,610 (1.46 MB)

 Trainable params: 381,610 (1.46 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8734 - loss: 0.3471 - val_accuracy: 0.8676 - val_loss: 0.3601
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8891 - loss: 0.3046 - val_accuracy: 0.8757 - val_loss: 0.3491
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8929 - loss: 0.2860 - val_accuracy: 0.8811 - val_loss: 0.3225
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9011 - loss: 0.2683 - val_accuracy: 0.8819 - val_loss: 0.3263
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9045 - loss: 0.2568 - val_accuracy: 0.8867 - val_loss: 0.3219
Epoch 6/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9081 - loss: 0.2444 - val_accuracy: 0.8913 - val_loss: 0.3149
Epoch 7/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9134 - loss: 0.2303 - val_accuracy: 0.8882 - val_loss: 0.3217
Epoch 8/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9172 - loss: 0.2217 - 

2. Tuner with more complex networks

In [14]:
# Create function build Hyper Model
def model_builder(hp):

  model = keras.Sequential()
  model.add(Flatten(input_shape=(28,28)))
  X1 = hp.Int("unit1", min_value=32, max_value=512, step=32)
  X2 = hp.Int("unit2", min_value=32, max_value=1024, step=32)
  X3 = hp.Int("unit3", min_value=32, max_value=256, step=32)
  Y = hp.Float("dr_value", min_value=0.2, max_value=0.3, step=0.05)
  
  model.add(Dense(units=X1, activation="relu"))
  model.add(Dense(units=X2, activation="relu"))
  model.add(Dropout(Y))
  model.add(Dense(units=X3, activation="relu"))
  model.add(Dense(10))

  lr = 0.001

  model.compile(optimizer=Adam(learning_rate=lr), loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
  return model


3. Demo GridSearch with SVM

In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC

In [16]:
# Load data avaiable in sklearn.dataset
cancer = load_breast_cancer()

In [17]:
# Convert data to DF
df_feat = pd.DataFrame(
  cancer['data'],
	columns = cancer['feature_names']
)

# cancer column is our target
df_target = pd.DataFrame(
  cancer['target'],
	columns =['Cancer']
)

print("Feature Variables: ")
print(df_feat.info())

Feature Variables: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smooth

In [18]:
df_feat.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [19]:
df_target.head()

,Cancer
0,0
1,0
2,0
3,0
4,0


In [21]:
# Chia train_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
	df_feat, 
  np.ravel(df_target),
  test_size = 0.30, 
  random_state = 101
)

In [22]:
# train the model
model = SVC()
model.fit(X_train, y_train)

# print results
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90        66
           1       0.91      0.97      0.94       105

    accuracy                           0.92       171
   macro avg       0.93      0.91      0.92       171
weighted avg       0.93      0.92      0.92       171



In [23]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {
  'C': [0.1, 1, 10, 100, 1000],
	'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
	'kernel': ['rbf']
}

grid=GridSearchCV(SVC(), param_grid, refit=True)

In [24]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']})

In [25]:
grid.best_params_

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}

In [26]:
model = grid.best_estimator_

In [27]:
# Dự đoán thử và in báo cáo
grid_predictions = model.predict(X_test)

# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91        66
           1       0.94      0.96      0.95       105

    accuracy                           0.94       171
   macro avg       0.94      0.93      0.93       171
weighted avg       0.94      0.94      0.94       171

